## Esercizio 52 - Diet problem

The goal of the diet problem is to select a set of foods that will satisfy a set of daily nutritional requirement at minimum cost. The problem is formulated as a integer program where the objective is to minimize cost and the constraints are to satisfy the specified nutritional requirements. The diet problem constraints typically regulate the number of calories and the amount of vitamins, minerals, fats, sodium, and cholesterol in the diet. While the mathematical formulation is simple, the solution may not be palatable! The nutritional requirements can be met without regard for taste or variety, so consider the output before digging into a meal from an "optimal" menu!

Given a set of foods $F$ and a set of nutrients $N$, along with the nutrient information for each food $\gamma_{ij}$ and the cost per serving of each food $u_i$, the objective of the diet problem is to select the number of servings of each food $x_i$ to purchase (and consume) so as to minimize the cost of the food while meeting the specified nutritional requirements. Typically, the nutritional requirements are expressed as a minimum $q^N_j$ and a maximum $Q^N_j$ allowable level for each nutritional component. Other constraints such a minimum $q^F_j$ and/or maximum number $Q^F_j$ of servings may be included to improve the quality of the menu.

\begin{align}
\min \quad & \sum\limits_{i \in F} u_i x_i &\\
s.t.\quad & q^N_j \leq \sum\limits_{i \in F} \gamma_{ij} x_i \leq Q^N_j  & j \in N \\
\quad & q^F_i \leq x_i \leq Q^F_i & i \in F\\
\quad & x_i \quad \mbox{integer} & i \in F
\end{align}

* TEST: diet.json

> Total cost = 2.87<br>
> Buy 2 of Spaghetti W/ Sauce<br>
> Buy 1 of Apple,Raw,W/Skin<br>
> Buy 10 of Chocolate Chip Cookies<br>
> Buy 2 of Lowfat Milk<br>
> Buy 1 of Hotdog<br>
> Total Calories = 2063.3<br>
> Total Calcium = 849.0<br>
> Total Iron = 11.3<br>
> Total Vit_A = 8201.9<br>
> Total Dietary_Fiber = 26.9<br>
> Total Carbohydrates = 272.0<br>
> Total Protein = 52.3

In [ ]:
# ALERT: execute this cell to prepare input data!
import requests
def download(link, nomeFile=None):
    if nomeFile == None:
        nomeFile = link.split('/')[-1]
    richiesta = requests.get(link)
    if richiesta.status_code == 200:
        with open(nomeFile, 'w') as file:
            file.write(richiesta.text)
            
download('https://tommasoadamo.it/data/diet.json')

In [ ]:
# ALERT: execute this cell to install DOcplex! 
!pip install docplex cplex

In [ ]:
import json
import docplex.mp.model as cplex
import matplotlib.pyplot as plt

with open('diet.json', 'r') as f:
    dati = json.load(f)

n = len(dati['FOODS'])
m = len(dati['NUTRIENTS'])

F = list(range(n))
N = list(range(m))

# dati sui foods
u = []
qF = []
QF = []
fnames = []

for food in dati['FOODS']:
    u.append(food['unit_cost'])
    qF.append(food['qmin'])
    QF.append(food['qmax'])
    fnames.append(food['name'])

# dati sui nutrients
qN = []
QN = []
nnames = []

for nutrient in dati['NUTRIENTS']:
    qN.append(nutrient['qmin'])
    QN.append(nutrient['qmax'])
    nnames.append(nutrient['name'])

gamma = dati['FOOD_NUTRIENTS']

with cplex.Model('diet') as mdl:
    x = mdl.integer_var_list(F, name='x')
    
    mdl.minimize(sum(u[i]*x[i] for i in F))
    
    for j in N:
        somma = sum(gamma[i][j]*x[i] for i in F)
        mdl.add_constraint(somma >= qN[j])
        mdl.add_constraint(somma <= QN[j])
    for i in F:
        mdl.add_constraint(x[i] >= qF[i])
        mdl.add_constraint(x[i] <= QF[i])
    
    mdl.print_information()
    sol = mdl.solve()
    if sol == None:
        print('No solution')
    else:
        print(sol)
        
        print('Total cost: ' + str(sol.objective_value))
        for i in F:
            if x[i].solution_value > 0:
                print('Buy {} of {}'.format(x[i].solution_value, fnames[i]))
                
        for j in N:
            somma = sum(gamma[i][j]*x[i].solution_value for i in F)
            print('Total {} = {}'.format(nnames[j], somma))
            
        valori = []
        for i in F:
            valori.append(x[i].solution_value)
        plt.pie(valori, labels=fnames, autopct='%1.1f%%', radius=2.5)
        plt.show()